In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
from sklearn.impute import KNNImputer as knn
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import mutual_info_regression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from statsmodels.stats.outliers_influence import variance_inflation_factor
data = pd.read_csv(r'/kaggle/input/playground-series-s4e4/train.csv')
data_pred=pd.read_csv(r'/kaggle/input/playground-series-s4e4/test.csv')

In [ ]:
!pip install -U pandas-profiling


In [ ]:
from ydata_profiling import ProfileReport
from ydata_profiling.utils.cache import cache_file
     

In [ ]:

profile = ProfileReport(
    data, title="Abalone Dataset", html={"style": {"full_width": True}}, sort=None
)
     

In [ ]:
data.drop('id',axis=1,inplace=True)

In [ ]:
class Utils_Suite():
    def __init__(self,data):
        self.data=data
    def compute_correlation(self,threshold=0.3):
        matrix=self.data.corr(numeric_only=True)
        x=matrix[(matrix["Rings"]<threshold)&(matrix["Rings"]>-threshold)]["Rings"]
        return x
    def compute_mutual_information(self,thresh=0.1):
        enc = OrdinalEncoder()
        df_encoded = enc.fit_transform(self.data)
        mi_scores = mutual_info_regression(df_encoded, self.data['Rings'])
        mi_scores_df = pd.DataFrame(mi_scores, index=self.data.columns, columns=['Score'])
        return mi_scores_df[mi_scores_df['Score']<thresh]
    def compute_vif(self):
        x=self.data.iloc[:,:-1]
        y=self.data.iloc[:,-1]
        x=pd.DataFrame(x)

        x['intercept']=1
        vif=pd.DataFrame()
        vif['variable']=x.columns
        vif['vif']=[variance_inflation_factor(x.values,i)for i in range(x.shape[1])]
        return vif

In [ ]:
def one_hot_encoding(data):
        z=(data.dtypes=='object')
        k=pd.DataFrame(z)
        obj_list=list(k[k[0]==True].index)
        print(obj_list)
        for i in obj_list:
            dummy=pd.get_dummies(data[i],prefix=i,drop_first=True)
            #print(dummy)
            data=data.drop(i,axis=1)
            data=data.join(dummy)
        return data
        

In [ ]:
def get_all_Null(data,dtype=""):
        x=data.isna().sum()>0
        l=[]
        for i in  list(x.index):
            thresh=data[i].isna().sum()/len(data)
            if(x[i]==True and (data[i].dtypes==dtype) ):
                print(i,data[i].isna().sum())
                l+=[i]
        return l

In [ ]:
def Label_Encoding(data1):
        data=data1
        enc=[]
        x=pd.DataFrame(data.dtypes)
        ll=list(x[x[0]=="object"].index)
        print(ll)
        for i in ll:
            label_encoder = LabelEncoder()
            label_encoder.fit(data[i])
            data[i]= label_encoder.transform(data[i])
            enc.append(label_encoder)
        return (data,enc)

In [ ]:

def StdScale(data1):
        data=data1
        for i in data.columns:
            if data[i].dtypes!='object' and i!='Rings':
                scale = StandardScaler().fit(data[[i]])

                data[i] = scale.transform(data[[i]])
        return data

In [ ]:
def drop_vif(data,thresh=5.5,col_Spare=['Rings','intercept']):


        vif=Utils_Suite(data).compute_vif()
        z1=vif[vif["vif"]>thresh]
        z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
        while True:
            try:
                col=z1.iloc[0,0]
                if z1.empty:
                    break
                if col in col_Spare:
                    z1=z1.iloc[1:]
                    continue
                data=data.drop(col,axis=1)
                vif=Utils_Suite(data).compute_vif()
                z1=vif[vif["vif"]>thresh]
                z1=z1.sort_values(by='vif', kind='mergesort',ascending=[False])
            except IndexError:
                break
        return data

In [ ]:
Utils_Suite(data).compute_correlation(threshold=0.9)

In [ ]:

def drop_correlation(data1):
        data=data1
        k=Utils_Suite(data).compute_correlation(0.3)
        f=pd.DataFrame(k)
        m=list(f[(f['Rings']<0.1) & (f['Rings']>-0.1)].index)
        data=data.drop(columns=m)
        print(m)
        return data
     

In [ ]:
data=drop_correlation(data)

In [ ]:
data=Label_Encoding(data)

In [ ]:
le=Label_Encoding(data)

In [ ]:
data=le[0]

In [ ]:
data=drop_vif(data)

In [ ]:
from sklearn.model_selection import GridSearchCV
param={
    
    'max_depth':[None,3,5,7],
    'bootstrap':[True,False],
    'max_features':['sqrt','log2']
  }
cls_grid=RandomForestRegressor(random_state=0)
gr=GridSearchCV(cls_grid,param_grid=param,cv=5)  # class model,param,crossval=5
gr.fit(x_train,y_train)
gr.best_params_

In [ ]:
data=StdScale(data)


In [ ]:
data

In [ ]:
from sklearn.model_selection import train_test_split
x=data.iloc[:,:-1].values
y=data.iloc[:,-1].values
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=2024)

In [ ]:
!pip install lazypredict

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
reg = GradientBoostingRegressor(n_estimators=300,max_depth=6,random_state=0)
reg.fit(x_train,y_train)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor( random_state=0)
regr.fit(x_train,y_train)

In [ ]:
y_pred=reg.predict(x_test)

In [ ]:
from sklearn.metrics import mean_squared_error0.17149515657483047,0.17112453268441297

In [ ]:
np.sqrt(mean_squared_error(np.log(y_test),np.log(y_pred)))

In [ ]:
#from lazypredict.Supervised import LazyRegressor
#reg = LazyRegressor(verbose=0, ignore_warnings=False, custom_metric=None)
#models, predictions = reg.fit(x_train, x_test, y_train, y_test)

#print(models)

In [ ]:
len(x_train)

In [ ]:
from xgboost import XGBRegressor


In [ ]:
model = XGBRegressor(
    max_depth=10,
    n_estimators=1000,
    min_child_weight=0.5, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    eta=0.1,
    seed=42)

In [ ]:
model.fit(
    x_train, 
    y_train, 
    eval_metric="rmse" )

In [ ]:
y_pred=model.predict(x_test)

In [ ]:
data_pred

In [ ]:
dd11=list(data.columns)
dd11.remove('Rings')
     

In [ ]:
data_pred=data_pred[dd11]


In [ ]:
data_pred

In [ ]:
data_pred=StdScale(data_pred)


In [ ]:
y_pred=reg.predict(data_pred.values)

In [ ]:
y_pred=pd.Series(y_pred)
ds=pd.concat([pd.Series(data_pred1['id'].values),y_pred],axis=1)
ds.columns=['id','Rings']
ds.to_csv('gradientboosting_regression1.csv',index=False)

In [ ]:
data_pred1=pd.read_csv(r'/kaggle/input/playground-series-s4e4/test.csv')

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
regr1 = AdaBoostRegressor(random_state=0, n_estimators=100)
regr1.fit(x_train,y_train)